Dataset

Training 
  Positive : 26974, 
  Negative : 27470

Validation 
  Positive : 2150, 
  Negative : 2136

In [ ]:
import json

import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# from tensorflow import feature_column
from tensorflow.keras import layers
# from sklearn.model_selection import train_test_split

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!ls '/content/drive/MyDrive/Colab Notebooks/2/training'

data-edit    training.txt	 Train_negative.txt   Train_positive.txt
train1.csv   training.xlsx	 Train_negative.xlsx
train1.xlsx  Train_negative.csv  Train_positive.csv


In [ ]:
#utitliy functions 

import os
import sys
# os.environ['MKL_THREADING_LAYER'] = 'GNU'
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import datetime

output_path="/content/drive/MyDrive/Colab Notebooks/2/results/TargetNet_training/"

def set_output(string):
    """ set output configurations """
    output, save_prefix = sys.stdout, None
    if output_path is not None:
        save_prefix = output_path
        if not os.path.exists(save_prefix):
            os.makedirs(save_prefix, exist_ok=True)
        output = open( output_path + string + ".txt", "a")

    return output, save_prefix

def Print(string, output, newline=False, timestamp=True):
    """ print to stdout and a file (if given) """
    if timestamp:
        time = datetime.datetime.now()
        line = '\t'.join([str(time.strftime('%m-%d %H:%M:%S')), string])
    else:
        time = None
        line = string

    print(line, file=sys.stderr)
    if newline: print("", file=sys.stderr)

    if not output == sys.stdout:
        print(line, file=output)
        if newline: print("", file=output)

    output.flush()
    return time

In [ ]:
!pip install bio
import sys
import numpy as np
from Bio import pairwise2

# import torch

#Loading the datasets
Positive_Data = '/content/drive/MyDrive/Colab Notebooks/2/data/miRAW_positive.csv'
Negative_Data = '/content/drive/MyDrive/Colab Notebooks/2/data/miRAW_negative.csv'
TestingData = '/content/drive/MyDrive/Colab Notebooks/2/data/csv/Independent_dataset1.csv'

class miRNA_CTS_dataset():
    """ Pytorch dataloader for miRNA-CTS pair data """
    def __init__(self, X, labels, set_idxs, set_labels):
        self.X = X
        self.labels = labels
        self.set_idxs = set_idxs
        self.set_labels = set_labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, i):
        return self.X[i], self.labels[i], self.set_idxs[i]

def encode_RNA(mirna_seq, mirna_esa, cts_rev_seq, cts_rev_esa):
    """ one-hot encoder for RNA sequences with/without extended seed alignments """
    chars = {"A":0, "C":1, "G":2, "U":3, "-":4}

    x = np.zeros((64, 64), dtype=np.float32)
    for i in range(len(mirna_esa)):
      for j in range(5):
        x[chars[mirna_esa[i]]+j, 5 + i] = 1
    for i in range(10, len(mirna_seq)):
      for j in range(5):
        x[chars[mirna_seq[i]]+j, 5 + i - 10 + len(mirna_esa)] = 1
    for i in range(5):
      for j in range(5):
        x[chars[cts_rev_seq[i]]+j + len(chars), i] = 1
    for i in range(len(cts_rev_esa)):
      for j in range(5):
        x[chars[cts_rev_esa[i]]+j + len(chars), i + 5] = 1
    for i in range(15, len(cts_rev_seq)):
      for j in range(5):
        x[chars[cts_rev_seq[i]]+j  + len(chars), i + 5 - 15 + len(cts_rev_esa)] = 1

    return x
        

# def encode_RNA(mirna_seq, mirna_esa, cts_rev_seq, cts_rev_esa):
#     """ one-hot encoder for RNA sequences with/without extended seed alignments """
#     chars = {"A":0, "C":1, "G":2, "U":3, "-":4}
 
#     x = np.zeros((len(chars)* 2, 50), dtype=np.float32)
#     for i in range(len(mirna_esa)):
#         x[chars[mirna_esa[i]], 5 + i] = 1
#     for i in range(10, len(mirna_seq)):
#         x[chars[mirna_seq[i]], 5 + i - 10 + len(mirna_esa)] = 1
#     for i in range(5):
#         x[chars[cts_rev_seq[i]] + len(chars), i] = 1
#     for i in range(len(cts_rev_esa)):
#         x[chars[cts_rev_esa[i]] + len(chars), i + 5] = 1
#     for i in range(15, len(cts_rev_seq)):
#         x[chars[cts_rev_seq[i]]  + len(chars), i + 5 - 15 + len(cts_rev_esa)] = 1

#     return x


def reverse(seq):
    """ reverse the given sequence """
    seq_r = ""
    for i in range(len(seq)):
        seq_r += seq[len(seq) - 1 - i]
    return seq_r


score_matrix = {}  # Allow watson-crick & wobble
for c1 in 'ACGU':
    for c2 in 'ACGU':
        if (c1, c2) in [('A', 'U'), ('U', 'A'), ('G', 'C'), ('C', 'G')]:
            score_matrix[(c1, c2)] = 1
        elif (c1, c2) in [('U', 'G'), ('G', 'U')]:
            score_matrix[(c1, c2)] = 1
        else:
            score_matrix[(c1, c2)] = 0


def extended_seed_alignment(mi_seq, cts_r_seq):
    """ extended seed alignment """
    alignment = pairwise2.align.globaldx(mi_seq[:10], cts_r_seq[5:15], score_matrix, one_alignment_only=True)[0]
    mi_esa = alignment[0]
    cts_r_esa = alignment[1]
    esa_score = alignment[2]
    return mi_esa, cts_r_esa, esa_score


def get_dataset_from_configs(data_cfg,split_idx=None):
  
    FILE = open(data_cfg, "r")
    lines = FILE.readlines()
    FILE.close()

    X, labels, set_idxs, set_labels = [], [], [], []
    set_idx = 0
    for l, line in enumerate(lines[1:]):
      tokens = line.strip().split("\t")
      print(tokens)
      mirna_id, mirna_seq, mrna_id, mrna_seq = tokens[:4]
      label = float(tokens[4]) if len(tokens) > 4 else 0
      
      mirna_seq = mirna_seq.upper().replace("T", "U")
      mrna_seq = mrna_seq.upper().replace("T", "U")
      mrna_rev_seq = reverse(mrna_seq)

      for pos in range(len(mrna_rev_seq) - 40 + 1):
          mirna_esa, cts_rev_esa, esa_score = extended_seed_alignment(mirna_seq, mrna_rev_seq[pos:pos+40])
          if split_idx not in ["train", "val"] and esa_score < 6: continue

          x = encode_RNA(mirna_seq, mirna_esa,mrna_rev_seq[pos:pos+40], cts_rev_esa).transpose()
          # x = np.array(np.repeat(x[..., np.newaxis], 1, -3))
          # print(x.shape)
          X.append(np.array(x))
          labels.append(tf.expand_dims(np.array(label), 0))
          set_idxs.append(tf.expand_dims(np.array(set_idx), 0))

      set_labels.append(label)
      set_idx += 1

      if set_idx % 5 == 0:
          print('# {} {:.1%}'.format(split_idx, l / len(lines[1:])), end='\r', file=sys.stderr)
    print(' ' * 150, end='\r', file=sys.stderr)

    # print(len(set_labels))
    # print((X[0]))
    # print(len(labels))
    X = np.array(X)
    labels = np.array(labels)
    dataset = miRNA_CTS_dataset(X, labels, set_idxs, np.array(set_labels))

    return X, labels, set_idxs, set_labels

output, save_prefix = set_output("train_model_log")
device = device_name

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 14.1 MB/s 
     |████████████████████████████████| 2.3 MB 57.2 MB/s 


In [ ]:
import random 

def shuffleData(X, y):
    index = [i for i in range(len(X))]
    random.shuffle(index)
    X = X[index]
    y = y[index]
    return X, y;

In [ ]:
def chunkIt(seq, num):
    avg = len(seq) / float(num)
    out = []
    last = 0.0

    while last < len(seq):
        out.append(seq[int(last):int(last + avg)])
        last += avg
    
    return out

In [ ]:
start = Print(" ".join(['start loading datasets']), output)

Positive_X, Positive_y, Positive_set_idxs, Positive_set_labels = get_dataset_from_configs(Positive_Data,split_idx="train")
Negative_X, Negative_y, Negative_set_idxs, Negative_set_labels = get_dataset_from_configs(Negative_Data, split_idx="val")

# ind_X, ind_y, ind_set_idxs, ind_set_labels = get_dataset_from_configs(Negative_Data, split_idx="val")

print(len(Positive_X))
print(len(Negative_X))

folds = 10
Positive_X_Slices = chunkIt(Positive_X, folds);
Positive_y_Slices = chunkIt(Positive_y, folds);

Negative_X_Slices = chunkIt(Negative_X, folds);
Negative_y_Slices = chunkIt(Negative_y, folds);

print(len(Positive_X_Slices), 'train examples')
print(len(Positive_y_Slices), 'validation examples')
print(len(Negative_X_Slices), 'test examples')


In [ ]:


# for test_index in range(folds):
#   test_X = np.concatenate((Positive_X_Slices[test_index],Negative_X_Slices[test_index]))
#   test_y = np.concatenate((Positive_y_Slices[test_index],Negative_y_Slices[test_index]))

#   validation_index = (test_index+1) % folds;

#   valid_X = np.concatenate((Positive_X_Slices[validation_index],Negative_X_Slices[validation_index]))
#   valid_y = np.concatenate((Positive_y_Slices[validation_index],Negative_y_Slices[validation_index]))

#   start = 0;

#   for val in range(0, folds):
#     if val != test_index and val != validation_index:
#       start = val;
#       break;

#   train_X = np.concatenate((Positive_X_Slices[start],Negative_X_Slices[start]))
#   train_y = np.concatenate((Positive_y_Slices[start],Negative_y_Slices[start]))

#   for i in range(0, folds):
#     if i != test_index and i != validation_index and i != start:
#       tempX = np.concatenate((Positive_X_Slices[i],Negative_X_Slices[i]))
#       tempy = np.concatenate((Positive_y_Slices[i],Negative_y_Slices[i]))

#       train_X = np.concatenate((train_X, tempX))
#       train_y = np.concatenate((train_y, tempy))

#   test_X, test_y = shuffleData(test_X,test_y);
#   valid_X,valid_y = shuffleData(valid_X,valid_y);
#   train_X,train_y = shuffleData(train_X,train_y);



In [ ]:
from keras.layers import Embedding, LSTM, Dense, Bidirectional
from keras import optimizers
from tensorflow import keras

drop_out = 0.3
LR= 0.001

def getModel(batch, lr, dout):
        model = Sequential()
        model.add(Conv1D(input_shape=(64, 64),filters=320,kernel_size=ksize,padding="same", activation="relu"))
        model.add(Conv1D(filters=320, kernel_size=ksize, activation='relu'))
        model.add(MaxPool1D(pool_size=2))
        # model.add(Conv1D(filters=320, kernel_size=ksize, activation='relu'))
        # model.add(MaxPool1D(pool_size=2))
        model.add(Dropout(dout))
        model.add(Bidirectional(LSTM(32, activation='relu')))
        model.add(Dropout(dout))
        model.add(Dense(16, activation='relu'))
        model.add(Dropout(dout))
        model.add(Dense(1, activation='sigmoid'))

        model.summary()
        adam = keras.optimizers.Adam(lr)
        model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
        
        return model

In [ ]:
from sklearn.metrics import confusion_matrix,recall_score,matthews_corrcoef,roc_curve,roc_auc_score,auc
#from tensorflow.compat.v1 import Session
from tensorflow.keras import losses

tf.compat.v1.disable_eager_execution()

def calculateScore(X, y, model):
    
    score = model.evaluate(X,y)
    pred_y = model.predict(X)

    accuracy = score[1];

    tempLabel = np.zeros(shape = y.shape, dtype=np.int32)

    for i in range(len(y)):
        if pred_y[i] < 0.5:
            tempLabel[i] = 0;
        else:
            tempLabel[i] = 1;
    confusion = confusion_matrix(y, tempLabel)
    TN, FP, FN, TP = confusion.ravel()

    sensitivity = recall_score(y, tempLabel)
    specificity = TN / float(TN+FP)
    MCC = matthews_corrcoef(y, tempLabel)

    F1Score = (2 * TP) / float(2 * TP + FP + FN)
    precision = TP / float(TP + FP)

    pred_y = pred_y.reshape((-1, ))

    ROCArea = roc_auc_score(y, pred_y)
    fpr, tpr, thresholds = roc_curve(y, pred_y)
    lossValue = None;

    print(y.shape)
    print(pred_y.shape)

    y_true = tf.convert_to_tensor(y, np.float32)
    y_pred = tf.convert_to_tensor(pred_y, np.float32)

    with tf.compat.v1.Session():
        lossValue = losses.binary_crossentropy(y_true, y_pred).eval()

    return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea, 'precision' : precision, 'F1' : F1Score, 'fpr' : fpr, 'tpr' : tpr, 'thresholds' : thresholds, 'lossValue' : lossValue}
    # return {'sn' : sensitivity, 'sp' : specificity, 'acc' : accuracy, 'MCC' : MCC, 'AUC' : ROCArea, 'precision' : precision, 'F1' : F1Score, 'lossValue' : lossValue}



In [ ]:
from scipy import interp
import matplotlib.pyplot as plt

def analyze(temp, OutputDir):

    # temp = None
    # with open(dataFile, 'rb') as file:
    #        temp = pickle.load(file)

    trainning_result, validation_result = temp;

    file = open(OutputDir + '/performance.txt', 'w')

    index = 0
    for x in [trainning_result, validation_result]:


        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'

        index += 1;

        file.write(title +  'results\n')


        for j in ['sn', 'sp', 'acc', 'MCC', 'AUC', 'precision', 'F1', 'lossValue']:

            total = []

            for val in x:
                total.append(val[j])

            file.write(j + ' : mean : ' + str(np.mean(total)) + ' std : ' + str(np.std(total))  + '\n')

        file.write('\n\n______________________________\n')
    file.close();

    index = 0

    for x in [trainning_result, validation_result]:

        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)

        i = 0

        for val in x:
            tpr = val['tpr']
            fpr = val['fpr']
            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            roc_auc = auc(fpr, tpr)
            aucs.append(roc_auc)
            plt.plot(fpr, tpr, lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.2f)' % (i+1, roc_auc))

            i += 1

        print;

        plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='Random', alpha=.8)

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        plt.plot(mean_fpr, mean_tpr, color='b',
                 label=r'Mean ROC (AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
                 lw=2, alpha=.8)

        std_tpr = np.std(tprs, axis=0)
        tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
        tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
        plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,
                         label=r'$\pm$ 1 std. dev.')

        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver operating characteristic curve')
        plt.legend(loc="lower right")

        title = ''

        if index == 0:
            title = 'training_'
        if index == 1:
            title = 'validation_'
      

        plt.savefig( OutputDir + '/' + title +'ROC.png')
        plt.close('all');

        index += 1;

In [ ]:
import tensorflow.compat.v1 as tf
from tensorflow.keras import regularizers, Input, Model, Sequential
from tensorflow.keras.models import load_model

from tensorflow.keras.layers import Conv1D, Conv2D, BatchNormalization, MaxPool1D, MaxPool2D, Flatten, Dropout, Dense
from tensorflow.keras.optimizers import SGD

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

tf.compat.v1.disable_eager_execution()

OutputDir = '/content/drive/MyDrive/Colab Notebooks/Research/Colab Notebooks/finals/results'

trainning_result = []
validation_result = []
testing_result = []

drop_out = 0.3
LR= 0.001
epochs = 50
# batches = [512, 256, 200, 100]
learning_rate = [0.001]
dropout = [0.3]
batches = [64]
# lr = 0.02
# dout = 0.2
fils = 320
ksize = 12
acc = 0

for test_index in range(folds):
  valid_X = np.concatenate((Positive_X_Slices[test_index],Negative_X_Slices[test_index]))
  valid_y = np.concatenate((Positive_y_Slices[test_index],Negative_y_Slices[test_index]))

  validation_index = (test_index+1) % folds;

  train_X = np.concatenate((Positive_X_Slices[validation_index],Negative_X_Slices[validation_index]))
  train_y = np.concatenate((Positive_y_Slices[validation_index],Negative_y_Slices[validation_index]))

  start = 0;

  for val in range(0, folds):
    if val != test_index and val != validation_index:
      start = val;
      break;

  train_X = np.concatenate((Positive_X_Slices[start],Negative_X_Slices[start]))
  train_y = np.concatenate((Positive_y_Slices[start],Negative_y_Slices[start]))

  for i in range(0, folds):
    if i != test_index and i != validation_index and i != start:
      tempX = np.concatenate((Positive_X_Slices[i],Negative_X_Slices[i]))
      tempy = np.concatenate((Positive_y_Slices[i],Negative_y_Slices[i]))

      train_X = np.concatenate((train_X, tempX))
      train_y = np.concatenate((train_y, tempy))

  # test_X, test_y = shuffleData(test_X,test_y);
  valid_X,valid_y = shuffleData(valid_X,valid_y);
  train_X,train_y = shuffleData(train_X,train_y);


  print("valid", valid_X.shape)
  print("valid", len(valid_X))

  print("train", train_X.shape)
  print("train", len(train_X))

  for batch in batches: 
    for lr in learning_rate:
      for dout in dropout:
        model = getModel(batch, lr, dout)

        early_stopping = EarlyStopping(monitor='accuracy', patience= 20)
        model_check = ModelCheckpoint(filepath = OutputDir + "/the_model"+ str(test_index+1) +".h5", monitor = 'accuracy', save_best_only=True)
        reduct_L_rate = ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=20)

        with tf.device('/device:GPU:0'):
          history = model.fit(train_X, train_y, batch_size = batch, epochs = epochs, validation_data = (valid_X, valid_y), callbacks = [model_check, early_stopping,reduct_L_rate])

        # bestModel =  load_model(OutputDir + "/the_model.h5")
      # score = bestModel.evaluate(test_X, test_y, verbose=0)
      # print("Accuracy: %.2f%%" % (score[1]*100))

      # print(calculateScore(train_X, train_y, bestModel))
      # print(calculateScore(valid_X, valid_y, bestModel))
      # print(calculateScore(test_X, test_y, bestModel))

        trainning_result.append(calculateScore(train_X, train_y, model));
        validation_result.append(calculateScore(valid_X, valid_y, model));
      # testing_result.append(calculateScore(test_X, test_y, model));

# temp_dict = (trainning_result, validation_result)
# analyze(temp_dict, OutputDir);

valid (5878, 64, 64)
valid 5878
train (47024, 64, 64)
train 47024
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 64, 320)           246080    
                                                                 
 conv1d_1 (Conv1D)           (None, 53, 320)           1229120   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 26, 320)          0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 26, 320)           0         
                                                                 
 bidirectional (Bidirectiona  (None, 64)               90368     
 l)                                                              
                                                        

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


47024/47024 [==============================] - 71s 2ms/sample - loss: 0.3230 - accuracy: 0.8639 - val_loss: 0.3194 - val_accuracy: 0.8300 - lr: 0.0010
Epoch 2/50
47024/47024 [==============================] - 52s 1ms/sample - loss: 0.2137 - accuracy: 0.9160 - val_loss: 0.3858 - val_accuracy: 0.8095 - lr: 0.0010
Epoch 3/50
47024/47024 [==============================] - 52s 1ms/sample - loss: 0.1770 - accuracy: 0.9327 - val_loss: 0.4038 - val_accuracy: 0.8049 - lr: 0.0010
Epoch 4/50
47024/47024 [==============================] - 52s 1ms/sample - loss: 0.1549 - accuracy: 0.9423 - val_loss: 0.4922 - val_accuracy: 0.7890 - lr: 0.0010
Epoch 5/50
47024/47024 [==============================] - 51s 1ms/sample - loss: 0.1360 - accuracy: 0.9501 - val_loss: 0.4255 - val_accuracy: 0.7902 - lr: 0.0010
Epoch 6/50
47024/47024 [==============================] - 52s 1ms/sample - loss: 0.1250 - accuracy: 0.9540 - val_loss: 0.4461 - val_accuracy: 0.7933 - lr: 0.0010
Epoch 7/50
47024/47024 [===============

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(47024, 1)
(47024,)
(5878, 1)
(5878,)


...

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2079: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


(47024, 1)
(47024,)
(5878, 1)
(5878,)


valid (5878, 64, 64)
valid 5878
train (47024, 64, 64)
train 47024
Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_18 (Conv1D)          (None, 64, 320)           246080    
                                                                 
 conv1d_19 (Conv1D)          (None, 53, 320)           1229120   
                                                                 
 max_pooling1d_9 (MaxPooling  (None, 26, 320)          0         
 1D)                                                             
                                                                 
 dropout_27 (Dropout)        (None, 26, 320)           0         
                                                                 
 bidirectional_9 (Bidirectio  (None, 64)               90368     
 nal)                                                            
                                                      

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


47024/47024 [==============================] - 55s 1ms/sample - loss: 0.3620 - accuracy: 0.8337 - val_loss: 0.1555 - val_accuracy: 0.9570 - lr: 0.0010
Epoch 2/50
47024/47024 [==============================] - 49s 1ms/sample - loss: 0.2428 - accuracy: 0.8986 - val_loss: 0.1264 - val_accuracy: 0.9599 - lr: 0.0010
Epoch 3/50
47024/47024 [==============================] - 49s 1ms/sample - loss: 0.2016 - accuracy: 0.9188 - val_loss: 0.1071 - val_accuracy: 0.9631 - lr: 0.0010
Epoch 4/50
47024/47024 [==============================] - 50s 1ms/sample - loss: 0.1770 - accuracy: 0.9309 - val_loss: 0.1054 - val_accuracy: 0.9639 - lr: 0.0010
Epoch 5/50
47024/47024 [==============================] - 50s 1ms/sample - loss: 0.1587 - accuracy: 0.9400 - val_loss: 0.1111 - val_accuracy: 0.9602 - lr: 0.0010
Epoch 6/50
47024/47024 [==============================] - 49s 1ms/sample - loss: 0.1459 - accuracy: 0.9437 - val_loss: 0.0947 - val_accuracy: 0.9668 - lr: 0.0010
Epoch 7/50
47024/47024 [===============

In [ ]:
temp_dict = (trainning_result, validation_result)
analyze(temp_dict, OutputDir);

In [ ]:
# import tensorflow.compat.v1 as tf
# from tensorflow.keras import regularizers, Input, Model, Sequential
# from tensorflow.keras.models import load_model

# from tensorflow.keras.layers import Conv1D, Conv2D, BatchNormalization, MaxPool1D, MaxPool2D, Flatten, Dropout, Dense
# from tensorflow.keras.optimizers import SGD

# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

# tf.compat.v1.disable_eager_execution()

# OutputDir = '/content/drive/MyDrive/Colab Notebooks/Research/Colab Notebooks/other_simlar_res_net/results'

# training_result = []
# validation_result = []
# testing_result = []

# epochs = 50
# batches = [512, 256, 200, 100]
# learning_rate = [0.2,0.1, 0.02, 0.01]
# dropout = [0.5, 0.4, 0.2]
# fils = 320
# ksize = 12
# acc = 0

# # model = getModel()
# for batch in batches: 
#   for lr in learning_rate:
#     for dout in dropout:
#       model = getModel(batch, lr, dout)

#       early_stopping = EarlyStopping(monitor='accuracy', patience=20)
#       model_check = ModelCheckpoint(filepath = OutputDir + "/model.h5", monitor = 'accuracy', save_best_only=True)
#       reduct_L_rate = ReduceLROnPlateau(monitor='val_loss',factor=0.1, patience=20)

#       with tf.device('/device:GPU:0'):
#         history = model.fit(train_X, train_y, batch_size = batch, epochs = epochs, validation_data = (valid_X, valid_y), callbacks = [model_check, early_stopping,reduct_L_rate])

      # bestModel = load_model( OutputDir + "/model.h5")
      # score = bestModel.evaluate(test_X, test_y, verbose=0)
      # print("Accuracy: %.2f%%" % (score[1]*100))

      # training_result.append(calculateScore(train_X, train_y, model))
      # validation_result.append(calculateScore(valid_X, valid_y, model))
      # testing_result.append(calculateScore(test_X, test_y, model));

      # temp_dict = (training_result, validation_result, testing_result)
      # analyze(temp_dict, OutputDir);